In [ ]:
!pip install kaggle

In [ ]:
# For Google Colab only
from google.colab import files
files.upload()  # Upload kaggle.json here

In [ ]:
# kaggle key activation

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Data retrieval
!kaggle datasets download -d sainikhileshreddy/food-recognition-2022
!unzip -q food-recognition-2022.zip -d dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

def show_images(images, titles=None, ncols=3):
    n = len(images)
    nrows = n // ncols + (1 if n % ncols else 0)
    fig, axs = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
    axs = axs.flatten()
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axs[i].imshow(img)
        axs[i].axis('off')
        if titles:
            axs[i].set_title(titles[i])
    plt.tight_layout()
    plt.show()

# Show first few images from a specific food category
food_category = './dataset/raw_data/public_training_set_release_2.0/images'  # Change as needed
image_files = [os.path.join(food_category, f) for f in os.listdir(food_category)[:9]]
show_images(image_files)